In [107]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-222138
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-222138


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "cluster01"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
if cluster_name not in ws.compute_targets:
    print("Creating the compute cluster with name:", cluster_name)
    compute_config=AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cluster=ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster=ws.compute_targets[cluster_name]
    print(cluster_name, ", compute cluster found. Using it ...")
cluster.wait_for_completion(show_output=True)

cluster01 , compute cluster found. Using it ...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os

# Specify parameter sampler

ps = RandomParameterSampling({"--C" : choice(0.001,0.01,0.1,1,10,20,100),"--max_iter": choice(50,100,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
est = SKLearn(source_directory = "./",
              entry_script = 'train.py',
              compute_target=cluster_name,
              vm_size = "Standard_D2_V2" )
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     estimator=est,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_a83df192-a48d-461d-8709-4859225abd90
Web View: https://ml.azure.com/runs/HD_a83df192-a48d-461d-8709-4859225abd90?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-222138/workspaces/quick-starts-ws-222138&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-10T11:15:06.104262][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-10T11:15:06.8099490Z][SCHEDULER][INFO]Scheduling job, id='HD_a83df192-a48d-461d-8709-4859225abd90_0' 
[2023-01-10T11:15:06.9535112Z][SCHEDULER][INFO]Scheduling job, id='HD_a83df192-a48d-461d-8709-4859225abd90_1' 
[2023-01-10T11:15:07.0141822Z][SCHEDULER][INFO]Scheduling job, id='HD_a83df192-a48d-461d-8709-4859225abd90_2' 
[2023-01-10T11:15:07.136767][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-01-10T11:15:07.1670823Z][SCHEDULER][INFO]Scheduling job, id='HD_a83df192-a48d-461d-8709-4859225abd

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_ba9520bf386d662001eeb9523395794e/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"train.py\", line 9, in <module>\n    import pandas as pd\nModuleNotFoundError: No module named 'pandas'\n\n Marking the experiment as failed because initial child jobs have failed due to user error",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process '/azureml-envs/azureml_ba9520bf386d662001eeb9523395794e/bin/python' exited with status code 1. Please check lo

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run =hyperdrive_run.get_best_run_by_primary_metric()

print("Best run metrics: ", best_run.get_metrics())
print("Best run details: ", best_rund.get_details())
print("Best run file names: ", best_rund.get_file_names())



In [72]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv' )

In [77]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [79]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [80]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d73228d9-3276-49f9-a7f1-02c0eadba1d3,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2023/01/10 10:05:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |yes                             |32950                                 |
+------------------------------+--------------------------------+-----

2023-01-10:10:31:29,131 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-01-10:10:31:31,175 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-01-10:10:31:31,177 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-01-10:10:31:33,71 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2023-01-10:10:31:33,73 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may 

StackEnsemble                                  0:00:56             0.9152    0.9166
Stopping criteria reached at iteration 32. Ending experiment.
********************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started


2023-01-10:10:33:45,292 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [85]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ####
best_run, fitted_model = automl_run.get_output()

#print(best_run)
#print(fitted_model)

print("Best run metrics: ", best_run.get_metrics())
print("Best run details: ", best_run.get_details())

#automl_best_run = automl_run.get_best_child()
#automl_best_run.get_details()

Best run metrics:  {'recall_score_weighted': 0.9165705614567526, 'average_precision_score_micro': 0.9815163208785542, 'matthews_correlation': 0.535806638472025, 'average_precision_score_macro': 0.8236077820136664, 'AUC_micro': 0.9807315245198387, 'f1_score_micro': 0.9165705614567526, 'AUC_macro': 0.9479093412301066, 'norm_macro_recall': 0.4702858522250326, 'precision_score_weighted': 0.9091045821509525, 'recall_score_micro': 0.9165705614567526, 'accuracy': 0.9165705614567526, 'recall_score_macro': 0.7351429261125163, 'f1_score_macro': 0.7636979470386827, 'average_precision_score_weighted': 0.9552127422255885, 'weighted_accuracy': 0.9616366155402989, 'f1_score_weighted': 0.9111664930966568, 'balanced_accuracy': 0.7351429261125163, 'precision_score_macro': 0.8054931179985353, 'precision_score_micro': 0.9165705614567526, 'log_loss': 0.30591377218110444, 'AUC_weighted': 0.9479093340607365, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_d73228d9-3276-49f9-a7f1-02c0eadba1d3_

In [ ]:
cluster01.delete()